In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity="all"

import os
import json
import argparse
import pandas as pd
import numpy as np
import time, datetime
from tqdm import tqdm
from logging import getLogger
import torch

from recbole.quick_start import run_recbole
import wandb

In [2]:
# train load
train = pd.read_csv("/opt/ml/input/data/train/train_ratings.csv")

# indexing save
user2idx = {v:k for k,v in enumerate(sorted(set(train.user)))}
item2idx = {v:k for k,v in enumerate(sorted(set(train.item)))}
uidx2user = {k:v for k,v in enumerate(sorted(set(train.user)))}
iidx2item = {k:v for k,v in enumerate(sorted(set(train.item)))}

In [3]:
train.user = train.user.map(user2idx)
train.item = train.item.map(item2idx)

train.columns=['user_id:token','item_id:token','timestamp:float']

outpath = f"dataset/train_data"
os.makedirs(outpath, exist_ok=True)
# sub_train=train.groupby("user").sample(n=10, random_state=SEED)
# sub_train.shape
train.to_csv(os.path.join(outpath,"train_data.inter"),sep='\t',index=False)

In [3]:
yamldata="""
USER_ID_FIELD: user_id
ITEM_ID_FIELD: item_id
TIME_FIELD: timestamp

load_col:
    inter: [user_id, item_id, timestamp]

show_progress : False
epochs : 10
device : torch.device("cuda" if torch.cuda.is_available() else "cpu")
eval_args:
    split: {'RS': [9, 1, 0]}
    group_by: user
    order: RO
    mode: full
metrics: ['Recall', 'MRR', 'NDCG', 'Hit', 'Precision', 'MAP']
topk: 10
valid_metric: Recall@10

log_wandb : True
wandb_project : Recbole

mlp_hidden_size : [600,400,200]
"""
with open("general.yaml", "w") as f:
    f.write(yamldata)

484

In [2]:
def run(model_name):
    if model_name in [
        "MultiVAE",
        "MultiDAE",
        "MacridVAE",
        "RecVAE",
        "GRU4Rec",
        "NARM",
        "STAMP",
        "NextItNet",
        "TransRec",
        "SASRec",
        "BERT4Rec",
        "SRGNN",
        "GCSAN",
        "GRU4RecF",
        "FOSSIL",
        "SHAN",
        "RepeatNet",
        "HRM",
        "NPE",
    ]:
        parameter_dict = {
            "neg_sampling": None,
        }
        return run_recbole(
            model=model_name,
            dataset='train_data',
            config_file_list=['general.yaml'],
            config_dict=parameter_dict,
        )
    else:
        return run_recbole(
            model=model_name,
            dataset='train_data',
            config_file_list=['general.yaml'],
        )

In [3]:
model_list = ['MultiVAE','MultiDAE'] # 해당 리스트에 쓰고 싶은 모델을 넣어줍니다.
os.environ['WANDB_NOTEBOOK_NAME'] = '/opt/ml/input/MR/Recbole/general/Recbole_general.ipynb'
for model_name in model_list:
    print(f"running {model_name}...")
    start = time.time()
    result = run(model_name)
    t = time.time() - start
    print(f"It took {t/60:.2f} mins")
    print(result)
    wandb.run.finish()

running MultiVAE...


27 Dec 05:17    INFO  ['/opt/conda/lib/python3.8/site-packages/ipykernel_launcher.py', '--ip=127.0.0.1', '--stdin=9008', '--control=9006', '--hb=9005', '--Session.signature_scheme="hmac-sha256"', '--Session.key=b"7fd8b746-1a87-4b32-97c0-fccd01819acb"', '--shell=9007', '--transport="tcp"', '--iopub=9009', '--f=/opt/ml/.local/share/jupyter/runtime/kernel-v2-48980F7V0o03cn7e6.json']
27 Dec 05:17    INFO  
General Hyper Parameters:
gpu_id = 0
use_gpu = True
seed = 2020
state = INFO
reproducibility = True
data_path = dataset/train_data
checkpoint_dir = saved
show_progress = False
save_dataset = False
dataset_save_path = None
save_dataloaders = False
dataloaders_save_path = None
log_wandb = True

Training Hyper Parameters:
epochs = 30
train_batch_size = 2048
learner = adam
learning_rate = 0.001
train_neg_sample_args = {'distribution': 'uniform', 'sample_num': 1, 'alpha': 1.0, 'dynamic': False, 'candidate_num': 0}
eval_step = 1
stopping_step = 10
clip_grad_norm = None
weight_decay = 0.0
loss_

27 Dec 05:18    INFO  epoch 0 training [time: 0.22s, train loss: 19151.7617]
27 Dec 05:19    INFO  epoch 0 evaluating [time: 49.00s, valid_score: 0.085700]
27 Dec 05:19    INFO  valid result: 
recall@10 : 0.0857    mrr@10 : 0.2913    ndcg@10 : 0.1364    hit@10 : 0.5899    precision@10 : 0.1146    map@10 : 0.0644
27 Dec 05:19    INFO  Saving current: saved/MultiVAE-Dec-27-2022_05-18-40.pth
27 Dec 05:19    INFO  epoch 1 training [time: 0.21s, train loss: 18392.2695]
27 Dec 05:20    INFO  epoch 1 evaluating [time: 48.13s, valid_score: 0.084500]
27 Dec 05:20    INFO  valid result: 
recall@10 : 0.0845    mrr@10 : 0.2989    ndcg@10 : 0.1373    hit@10 : 0.5878    precision@10 : 0.1135    map@10 : 0.0653
27 Dec 05:20    INFO  epoch 2 training [time: 0.21s, train loss: 18375.4537]
27 Dec 05:21    INFO  epoch 2 evaluating [time: 49.35s, valid_score: 0.087300]
27 Dec 05:21    INFO  valid result: 
recall@10 : 0.0873    mrr@10 : 0.3115    ndcg@10 : 0.1427    hit@10 : 0.5926    precision@10 : 0.1167

It took 26.96 mins
{'best_valid_score': 0.1382, 'valid_score_bigger': True, 'best_valid_result': OrderedDict([('recall@10', 0.1382), ('mrr@10', 0.4346), ('ndcg@10', 0.22), ('hit@10', 0.7649), ('precision@10', 0.1823), ('map@10', 0.1157)]), 'test_result': None}


train/epoch,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇███
train/train_loss,█▆▆▆▅▅▅▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▂▂▂▁▁▁▁▁
train_step,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇███
valid/hit@10,▁▁▁▁▁▂▂▂▃▃▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇███
valid/map@10,▁▁▂▁▁▁▂▂▃▃▄▄▄▄▅▅▅▅▅▆▆▆▇▆▇█▇███
valid/mrr@10,▁▁▂▂▂▂▂▃▃▄▄▄▅▄▅▆▆▆▆▆▆▇▇▇▇█████
valid/ndcg@10,▁▁▂▁▁▁▂▂▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▆▇█▇███
valid/precision@10,▁▁▁▁▁▁▂▂▃▃▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇███
valid/recall@10,▁▁▁▁▁▂▂▂▃▃▄▄▄▄▅▅▅▅▆▆▆▆▆▆▇▇▇███
valid_step,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇███
eval/hit@10,0.7649


running MultiDAE...


27 Dec 05:44    INFO  ['/opt/conda/lib/python3.8/site-packages/ipykernel_launcher.py', '--ip=127.0.0.1', '--stdin=9008', '--control=9006', '--hb=9005', '--Session.signature_scheme="hmac-sha256"', '--Session.key=b"7fd8b746-1a87-4b32-97c0-fccd01819acb"', '--shell=9007', '--transport="tcp"', '--iopub=9009', '--f=/opt/ml/.local/share/jupyter/runtime/kernel-v2-48980F7V0o03cn7e6.json']
27 Dec 05:44    INFO  
General Hyper Parameters:
gpu_id = 0
use_gpu = True
seed = 2020
state = INFO
reproducibility = True
data_path = dataset/train_data
checkpoint_dir = saved
show_progress = False
save_dataset = False
dataset_save_path = None
save_dataloaders = False
dataloaders_save_path = None
log_wandb = True

Training Hyper Parameters:
epochs = 30
train_batch_size = 2048
learner = adam
learning_rate = 0.001
train_neg_sample_args = {'distribution': 'uniform', 'sample_num': 1, 'alpha': 1.0, 'dynamic': False, 'candidate_num': 0}
eval_step = 1
stopping_step = 10
clip_grad_norm = None
weight_decay = 0.0
loss_

27 Dec 05:45    INFO  epoch 0 training [time: 0.20s, train loss: 19194.7581]
27 Dec 05:46    INFO  epoch 0 evaluating [time: 49.39s, valid_score: 0.085800]
27 Dec 05:46    INFO  valid result: 
recall@10 : 0.0858    mrr@10 : 0.2954    ndcg@10 : 0.1372    hit@10 : 0.5922    precision@10 : 0.1148    map@10 : 0.0648
27 Dec 05:46    INFO  Saving current: saved/MultiDAE-Dec-27-2022_05-45-34.pth
27 Dec 05:46    INFO  epoch 1 training [time: 0.20s, train loss: 18393.0508]
27 Dec 05:47    INFO  epoch 1 evaluating [time: 49.79s, valid_score: 0.086900]
27 Dec 05:47    INFO  valid result: 
recall@10 : 0.0869    mrr@10 : 0.2984    ndcg@10 : 0.139    hit@10 : 0.5961    precision@10 : 0.116    map@10 : 0.0661
27 Dec 05:47    INFO  Saving current: saved/MultiDAE-Dec-27-2022_05-45-34.pth
27 Dec 05:47    INFO  epoch 2 training [time: 0.20s, train loss: 18373.1202]
27 Dec 05:48    INFO  epoch 2 evaluating [time: 49.83s, valid_score: 0.088000]
27 Dec 05:48    INFO  valid result: 
recall@10 : 0.088    mrr@

It took 25.99 mins
{'best_valid_score': 0.135, 'valid_score_bigger': True, 'best_valid_result': OrderedDict([('recall@10', 0.135), ('mrr@10', 0.4366), ('ndcg@10', 0.219), ('hit@10', 0.7548), ('precision@10', 0.1801), ('map@10', 0.1158)]), 'test_result': None}


train/epoch,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇███
train/train_loss,█▅▅▅▅▄▄▄▄▄▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▁▁▁▁▁
train_step,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇███
valid/hit@10,▁▁▁▁▁▂▂▂▃▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇███
valid/map@10,▁▁▂▁▁▂▂▂▃▃▄▄▃▄▄▄▄▅▅▅▅▆▆▆▆▆▇███
valid/mrr@10,▁▁▂▂▂▂▂▃▃▃▄▄▄▄▅▄▅▅▅▅▆▆▆▆▇▇▇███
valid/ndcg@10,▁▁▁▁▁▂▂▂▃▃▄▄▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇███
valid/precision@10,▁▁▁▁▁▂▂▂▃▃▄▄▃▄▄▄▄▅▅▅▆▆▆▆▆▇▇███
valid/recall@10,▁▁▁▁▁▂▂▂▃▃▄▄▄▄▄▄▅▅▅▅▆▆▆▆▆▇▇███
valid_step,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇███
eval/hit@10,0.7548


### Model Load

In [ ]:
model_path='./saved/EASE-Dec-26-2022_18-01-48.pth'
# rank K 설정
K = 10

### Inference

In [ ]:
# config, model, dataset 불러오기
checkpoint = torch.load(model_path)
config = checkpoint['config']
config['dataset'] = 'train_data'

dataset = create_dataset(config)
train_data, valid_data, test_data = data_preparation(config, dataset)

model = get_model(config['model'])(config, test_data.dataset).to(config['device'])
model.load_state_dict(checkpoint['state_dict'])
model.load_other_parameter(checkpoint.get('other_parameter'))

# device 설정
device = config.final_config_dict['device']

# user, item id -> token 변환 array
user_id = config['USER_ID_FIELD']
item_id = config['ITEM_ID_FIELD']
user_id2token = dataset.field2id_token[user_id]
item_id2token = dataset.field2id_token[item_id]

# user id list
all_user_list = torch.arange(1, len(user_id2token)).view(-1,128) # 245, 128

# user, item 길이
user_len = len(user_id2token) # 31361 (PAD 포함)
item_len = len(item_id2token) # 6808 (PAD 포함)

# user-item sparse matrix
matrix = dataset.inter_matrix(form='csr') # (31361, 6808)

# user id, predict item id 저장 변수
pred_list = None
user_list = None

# model 평가모드 전환
model.eval()

# progress bar 설정
tbar = tqdm(all_user_list, desc=set_color(f"Inference", 'pink')) # 245, 128

for data in tbar: # data: 128, 
    # interaction 생성
    interaction = dict()
    interaction = Interaction(interaction)
    interaction[user_id] = data
    interaction = interaction.to(device)

    # user item별 score 예측
    score = model.full_sort_predict(interaction) # [1, 871424]
    score = score.view(-1, item_len) # 128, 6808

    rating_pred = score.cpu().data.numpy().copy() # 128, 6808

    user_index = data.numpy() # 128,

    # idx에는 128명의 영화상호작용이 True, False로 있다.
    idx = matrix[user_index].toarray() > 0 # idx shape: 128, 6808

    rating_pred[idx] = -np.inf # idx에서 True부분이 -inf로 변경
    rating_pred[:, 0] = -np.inf # 첫번째 PAD 열도 -inf로 변경
    
    # np.argpartition(배열, -K) : 배열에서 순서 상관없이 큰 값 K개를 뽑아 오른쪽에 놓겠다 -> 인덱스반환
    # rating_pred에서 각 행마다 K개의 score가 큰 인덱스를 오른쪽에 두고, 그 K개만 가져오기
    ind = np.argpartition(rating_pred, -K)[:, -K:] # rating_pred: (128, 6808) -> ind: (128, 20)

    user_row_index = np.arange(len(rating_pred)).reshape(-1,1) # [[0],[1],...,[127]]
    arr_ind = rating_pred[user_row_index, ind] # 128, 6808 -> 128, 20

    # arr_ind 내부에서 행별로, 내림차순 정렬해서 index 나오도록
    arr_ind_argsort = np.argsort(arr_ind)[np.arange(len(rating_pred)), ::-1]

    # ind는 item의 real index를 갖는 128,20 -> arr_ind_argsort를 통해 pred가 높은 상위 20개 read index 추출
    batch_pred_list = ind[user_row_index, arr_ind_argsort] # 128,20 -> 128,20

    if pred_list is None: # 처음에는 직접 정의
        pred_list = batch_pred_list
        user_list = user_index
    else: # pred_list가 있을 때는, append
        pred_list = np.append(pred_list, batch_pred_list, axis=0)
        user_list = np.append(
            user_list, user_index, axis=0
        )

result = []
for user, pred in zip(user_list, pred_list):
    for item in pred:
        result.append((int(user_id2token[user]), int(item_id2token[item])))

# 데이터 저장
sub = pd.DataFrame(result, columns=["user", "item"])
sub.to_csv(
    "submission.csv", index=False
)
print('inference done!')

### Submission

In [ ]:
sub = pd.read_csv('submission.csv')
sub.user = sub.user.map(uidx2user)
sub.item = sub.item.map(iidx2item)
sub.to_csv('model_version.csv',index=False)